In [ ]:
import os
import json
import requests
import pandas as pd
from pathlib import Path
from datetime import datetime
import time
from astropy.table import Table
import astropy.units as u
from astropy.cosmology import Planck18, WMAP9

In [ ]:
DATA_DIR = Path("../data/raw")
DATA_DIR.mkdir(parents=True, exist_ok=True)

In [ ]:
NASA_EXOPLANET_ARCHIVE = "https://exoplanetarchive.ipac.caltech.edu/TAP/sync"
NED_API = "https://ned.ipac.caltech.edu/srs/Query"
SIMBAD_API = "http://simbad.u-strasbg.fr/simbad/sim-tap/sync"
COSMOLIB_API = "https://cosmologist.info/cosmobox/cosmobox_interface.py"

In [ ]:
def fetch_nasa_exoplanet_data():
    """Fetch exoplanet data from NASA Exoplanet Archive."""
    print("Fetching NASA Exoplanet Archive data...")
    try:
        # Query for confirmed exoplanets with basic parameters
        query = """
        SELECT pl_name, hostname, pl_orbper, pl_rade, pl_bmasse, pl_orbsmax, 
               st_teff, st_rad, st_mass, st_met, st_metratio, st_age, sy_dist
        FROM ps
        WHERE default_flag = 1
        """
        
        params = {
            'query': query,
            'format': 'json'
        }
        
        response = requests.get(NASA_EXOPLANET_ARCHIVE, params=params)
        response.raise_for_status()
        
        data = response.json()
        df = pd.DataFrame(data)
        
        # Save to CSV
        output_file = DATA_DIR / 'nasa_exoplanets.csv'
        df.to_csv(output_file, index=False)
        print(f"✓ NASA Exoplanet data saved to {output_file}")
        return df
        
    except Exception as e:
        print(f"Error fetching NASA Exoplanet data: {e}")
        return None

In [ ]:
def fetch_ned_galaxy_data():
    """Fetch galaxy data from NASA/IPAC Extragalactic Database."""
    print("Fetching NED galaxy data...")
    try:
        # Example query for nearby galaxies with redshift < 0.1
        query = {
            'search_type': 'Search',
            'of': 'xml_main',
            'obj_type': 'Galaxy',
            'z_constraint': 'Unconstrained',
            'z_value1': '0',
            'z_value2': '0.1',
            'z_unit': 'z',
            'ot_include': 'ANY',
            'nmp_op': 'ANY',
            'out_csys': 'Equatorial',
            'out_equinox': 'J2000.0',
            'obj_sort': 'RA or Longitude',
            'of': 'csv_table',
            'zv_breaker': 30000.0,
            'list_limit': 1000,
            'img_stamp': 'NO',
            'search_form': 'Search form'
        }
        
        response = requests.get(NED_API, params=query)
        response.raise_for_status()
        
        # Save raw response
        output_file = DATA_DIR / 'ned_galaxies.csv'
        with open(output_file, 'w', encoding='utf-8') as f:
            f.write(response.text)
        print(f"✓ NED galaxy data saved to {output_file}")
        
        # Return as DataFrame
        return pd.read_csv(output_file)
        
    except Exception as e:
        print(f"Error fetching NED data: {e}")
        return None

In [ ]:
def fetch_simbad_objects():
    """Fetch astronomical objects from SIMBAD."""
    print("Fetching SIMBAD data...")
    try:
        # ADQL query for SIMBAD TAP service
        query = """
        SELECT TOP 1000
            main_id, ra, dec, otype_txt, sp_type, 
            rvz_redshift as z, flux, flux_bibcode
        FROM basic
        WHERE ra IS NOT NULL 
        AND dec IS NOT NULL
        AND otype_txt IN ('Galaxy', 'QSO', 'Seyfert', 'LINER')
        """
        
        params = {
            'request': 'doQuery',
            'lang': 'adql',
            'format': 'json',
            'query': query
        }
        
        response = requests.get(SIMBAD_API, params=params)
        response.raise_for_status()
        
        data = response.json()
        df = pd.DataFrame(data['data'], columns=[col['name'] for col in data['metadata']])
        
        # Save to CSV
        output_file = DATA_DIR / 'simbad_objects.csv'
        df.to_csv(output_file, index=False)
        print(f"✓ SIMBAD data saved to {output_file}")
        return df
        
    except Exception as e:
        print(f"Error fetching SIMBAD data: {e}")
        return None


In [ ]:
def fetch_cosmological_parameters():
    """Fetch current best-fit cosmological parameters."""
    print("Fetching cosmological parameters...")
    try:
        # Planck 2018 results
        planck = {
            'parameter': ['H0', 'Omega_m', 'Omega_Lambda', 'Omega_b', 'sigma_8', 'n_s', 'tau'],
            'value': [67.36, 0.3153, 0.6847, 0.0493, 0.8111, 0.9649, 0.0544],
            'error': [0.54, 0.0073, 0.0073, 0.0006, 0.0060, 0.0042, 0.0073],
            'unit': ['km/s/Mpc', '1', '1', '1', '1', '1', '1'],
            'source': ['Planck 2018'] * 7
        }
        
        # SH0ES 2023 results
        sh0es = {
            'parameter': ['H0'],
            'value': [73.04],
            'error': [1.04],
            'unit': ['km/s/Mpc'],
            'source': ['SH0ES 2023']
        }
        
        # Combine data
        df_planck = pd.DataFrame(planck)
        df_sh0es = pd.DataFrame(sh0es)
        df_cosmo = pd.concat([df_planck, df_sh0es], ignore_index=True)
        
        # Add timestamp
        df_cosmo['timestamp'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        
        # Save to CSV
        output_file = DATA_DIR / 'cosmological_parameters.csv'
        df_cosmo.to_csv(output_file, index=False)
        print(f"✓ Cosmological parameters saved to {output_file}")
        return df_cosmo
        
    except Exception as e:
        print(f"Error fetching cosmological parameters: {e}")
        return None

In [ ]:
def main():
    """Main function to fetch all data."""
    print("Starting API data fetching...")
    
    # Fetch data from different sources
    nasa_data = fetch_nasa_exoplanet_data()
    ned_data = fetch_ned_galaxy_data()
    simbad_data = fetch_simbad_objects()
    cosmo_data = fetch_cosmological_parameters()
    
    print("\nAPI data fetching complete!")
    
    # Print summary
    print("\nSummary of fetched data:")
    if nasa_data is not None:
        print(f"- NASA Exoplanet Archive: {len(nasa_data)} exoplanets")
    if ned_data is not None:
        print(f"- NED: {len(ned_data)} galaxies")
    if simbad_data is not None:
        print(f"- SIMBAD: {len(simbad_data)} objects")
    if cosmo_data is not None:
        print(f"- Cosmological parameters: {len(cosmo_data)} parameters")

if __name__ == "__main__":
    main()